# Pre-processing Utah Allocation data for WaDEQA upload.
- Purpose:  To pre-process the Utah data into one master file for simple DataFrame creation and extraction

In [1]:
#Needed Libararies
import os
import re
import numpy as np
import pandas as pd
import geopandas as gpd # the library that lets us read in shapefiles
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebooki

# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Utah/WaterAllocation/RawInputData"
os.chdir(workingDir)

## Point of Diversion Data

In [2]:
# Input File
FI_PoD = "PointsOfDiversion_input.csv"
dfPODin = pd.read_csv(FI_PoD, encoding = "ISO-8859-1")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfPODin:
    dfPODin['WaDEUUID'] = "utD" + dfPODin.index.astype(str)
    dfPODin.to_csv('PointsOfDiversion_input.csv', index=False)

dfPODin.head()

,Ã¯Â»Â¿OID_,OBJECTID,WRNUM,CHEXNUM,TYPE,SUMMARY_ST,STATUS,PRIORITY,USES,CFS,ACFT,LOCATION,WIN,OWNER,SOURCE,WebLink,Latitude,Longitude,WaDEUUID
0,0,795590894,0005007P00,,Underground,A,APPLAPP,,,0.0,0.0,N460 E1460 SW 17 25S 23E SL,22579,CASTLE VALLEY TOWN OF,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.624341,-109.402417,utD0
1,1,795590895,0005008P00,,Underground,A,APPLAPP,,,0.0,0.0,S100 E1650 NW 20 25S 23E SL,22558,CASTLE VALLEY TOWN OF,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.622795,-109.401786,utD1
2,2,795590896,0005009P00,,Underground,A,APPLAPP,,,0.0,0.0,S1310 E1160 NW 07 25S 23E SL,22794,CASTLE VALLEY TOWN OF,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.648267,-109.422402,utD2
3,3,795590897,0005010P00,,Underground,A,APPLAPP,,,0.0,0.0,N640 W1572 E4 26 26S 22E SL,22906,SPANISH VALLEY WATER SEWER IMPROVEMENT DISTRICT,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.514316,-109.465410,utD3
4,4,795590898,0009001M00,,Underground,A,APPLAPP,,,0.0,0.0,S1600 W2000 NE 33 37S 22E SL,22026,INTERNATIONAL URANIUM (USA) CORP.,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=000...,37.529540,-109.501160,utD4


In [3]:
def fixNativeID(valA):
    outString = str(valA).strip()
    return outString

dfPODin['WRNUM'] = dfPODin.apply(lambda row: fixNativeID(row['WRNUM']), axis=1)

In [4]:
# For creating BeneficialUseCategory
benUseDict = {
    "I" : "Irrigation",
    "S" : "Stockwatering",
    "D" : "Domestic",
    "M" : "Municipal",
    "X" : "Mining",
    "P" : "Power",
    "O" : "Other"}
def assignBenUseCategory(colrowValue):
    colrowValue = str(colrowValue).strip()
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        outList = ",".join(benUseDict[inx] for inx in list(str(colrowValue)))
    return outList


dfPODin['in_BeneficialUseCategory'] = dfPODin.apply(lambda row: assignBenUseCategory(row['USES']), axis=1)
dfPODin['in_BeneficialUseCategory'].unique()

array(['Unspecified', 'Domestic,Irrigation,Stockwatering', 'Domestic',
       'Stockwatering', 'Domestic,Stockwatering', 'Other',
       'Domestic,Irrigation,Other,Stockwatering', 'Other,Stockwatering',
       'Irrigation', 'Domestic,Irrigation',
       'Irrigation,Other,Stockwatering', 'Domestic,Other', 'Municipal',
       'Mining', 'Irrigation,Stockwatering', 'Irrigation,Other,Power',
       'Irrigation,Other', 'Domestic,Irrigation,Municipal,Stockwatering',
       'Domestic,Irrigation,Other', 'Domestic,Mining', 'Power',
       'Domestic,Irrigation,Municipal,Other',
       'Domestic,Irrigation,Municipal,Other,Stockwatering',
       'Irrigation,Municipal', 'Other,Mining',
       'Domestic,Stockwatering,Power',
       'Irrigation,Municipal,Other,Stockwatering',
       'Domestic,Other,Stockwatering',
       'Domestic,Irrigation,Stockwatering,Mining',
       'Irrigation,Municipal,Other', 'Domestic,Municipal,Other',
       'Municipal,Other', 'Domestic,Irrigation,Other,Power',
       'Domes

In [5]:
# Creating the output Dataframe for PODs.

dfPOD = pd.DataFrame(index=dfPODin.index)

# Water Source
dfPOD["in_WaterSourceTypeCV"] = dfPODin['TYPE']

# Site
dfPOD["in_Latitude"] = dfPODin['Latitude']
dfPOD["in_Longitude"] = dfPODin['Longitude']
dfPOD["in_PODorPOUSite"] = "POD"
dfPOD["in_SiteName"] = dfPODin['SOURCE']
dfPOD["in_SiteNativeID"] = "POD" + dfPODin['OBJECTID'].astype(str)
dfPOD["in_SiteTypeCV"] = dfPODin['SOURCE']

# Allocation
dfPOD["in_AllocationFlow_CFS"] = dfPODin['CFS'].astype(float)
dfPOD["in_AllocationVolume_AF"] = dfPODin['ACFT'].astype(float)
dfPOD['in_AllocationLegalStatusCV'] = dfPODin['STATUS']
dfPOD["in_AllocationNativeID"] = dfPODin['WRNUM']
dfPOD['in_AllocationOwner'] = dfPODin['OWNER']
dfPOD['in_AllocationPriorityDate'] = dfPODin['PRIORITY']
dfPOD["in_BeneficialUseCategory"] = dfPODin['in_BeneficialUseCategory']  #see above
dfPOD["in_IrrigatedAcreage"] = ""

dfPOD = dfPOD.drop_duplicates().reset_index(drop=True)
print(len(dfPOD))
dfPOD.head(1)

350411


,in_WaterSourceTypeCV,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_AllocationFlow_CFS,in_AllocationVolume_AF,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_BeneficialUseCategory,in_IrrigatedAcreage
0,Underground,38.624341,-109.402417,POD,Non-Production Well: Test,POD795590894,Non-Production Well: Test,0.0,0.0,APPLAPP,0005007P00,CASTLE VALLEY TOWN OF,,Unspecified,


## Place of Use Data

In [6]:
# Input File
FI_POU = "Utah_Place_of_Use_Irrigation_input.csv"
dfPOUin = pd.read_csv(FI_POU, encoding = "ISO-8859-1") # Place of Use Input

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfPOUin:
    dfPOUin['WaDEUUID'] = "utU" + dfPOUin.index.astype(str)
    dfPOUin.to_csv('Utah_Place_of_Use_Irrigation_input.csv', index=False)

dfPOUin.head()

C:\Users\rjame\AppData\Local\Temp\ipykernel_8724\4126468308.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfPOUin = pd.read_csv(FI_POU, encoding = "ISO-8859-1") # Place of Use Input


,Ã¯Â»Â¿OID_,RECORD_ID,GROUP_NUMB,AREA_CODE,CHNUM,POU_TYPE,SOURCE,URL,ACRES,WRNUMS,dbURL,recordID,Shape__Are,Shape__Len,Latitude,Longitude,WaDEUUID
0,0,21320841,85,1,,,ProofMap,https://waterrights.utah.gov/adjdinfo/hydromap...,0.004309,01-1127,https://maps.waterrights.utah.gov/POUPolygons/...,1,17.431641,31.168589,38.655222,-109.673950,utU0
1,1,21320842,82,1,,,ProofMap,https://waterrights.utah.gov/docImport/0547/05...,18.945065,"01-1078, 01-1124",https://maps.waterrights.utah.gov/POUPolygons/...,3,76631.458008,2248.417017,38.658167,-109.684943,utU1
2,2,21320843,80,1,,,ProofMap,https://waterrights.utah.gov/docImport/0547/05...,0.493870,01-1122,https://maps.waterrights.utah.gov/POUPolygons/...,4,1998.119629,212.278953,38.849461,-109.283069,utU2
3,3,21320844,627071,1,,,ProofMap,https://waterrights.utah.gov/docImport/0547/05...,0.167299,01-1169,https://maps.waterrights.utah.gov/POUPolygons/...,5,676.938477,433.033343,39.072978,-109.131510,utU3
4,4,21320845,63,1,,,ProofMap,https://waterrights.utah.gov/docSys/v903/K903/...,0.420000,01-1106,https://maps.waterrights.utah.gov/POUPolygons/...,28,1838.147217,411.681126,38.679793,-109.687282,utU4


In [7]:
# Need to split out WRNUMS into their own row
# The explode() method explodes lists into separate rows.
dfPOUin = dfPOUin.assign(WRNUMS=dfPOUin['WRNUMS'].str.split(',')).explode('WRNUMS').reset_index()
dfPOUin = dfPOUin.rename({'WRNUMS': 'WRNUM'}, axis=1)
dfPOUin = dfPOUin.replace(np.nan, "").reset_index()
print(len(dfPOUin))
dfPOUin.head(1)

214503


,level_0,index,Ã¯Â»Â¿OID_,RECORD_ID,GROUP_NUMB,AREA_CODE,CHNUM,POU_TYPE,SOURCE,URL,ACRES,WRNUM,dbURL,recordID,Shape__Are,Shape__Len,Latitude,Longitude,WaDEUUID
0,0,0,0,21320841,85,1,,,ProofMap,https://waterrights.utah.gov/adjdinfo/hydromap...,0.004309,01-1127,https://maps.waterrights.utah.gov/POUPolygons/...,1,17.431641,31.168589,38.655222,-109.67395,utU0


In [8]:
def fixNativeID(valA):
    outString = str(valA).strip()
    return outString

dfPOUin['WRNUM'] = dfPOUin.apply(lambda row: fixNativeID(row['WRNUM']), axis=1)

In [9]:
# CFS not provided for POU data.  Will instead assume they share values.
# merging dfPOD data to ensure that the POUs are using the same CFS and AF as the PODS.
dfPOUin = pd.merge(dfPOUin, dfPOD, left_on='WRNUM', right_on='in_AllocationNativeID', how='left')
print(len(dfPOUin))
dfPOUin.head(1)

780913


,level_0,index,Ã¯Â»Â¿OID_,RECORD_ID,GROUP_NUMB,AREA_CODE,CHNUM,POU_TYPE,SOURCE,URL,ACRES,WRNUM,dbURL,recordID,Shape__Are,Shape__Len,Latitude,Longitude,WaDEUUID,in_WaterSourceTypeCV,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_AllocationFlow_CFS,in_AllocationVolume_AF,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_BeneficialUseCategory,in_IrrigatedAcreage
0,0,0,0,21320841,85,1,,,ProofMap,https://waterrights.utah.gov/adjdinfo/hydromap...,0.004309,01-1127,https://maps.waterrights.utah.gov/POUPolygons/...,1,17.431641,31.168589,38.655222,-109.67395,utU0,Underground,38.655778,-109.672028,POD,Underground Water Well,POD795594373,Underground Water Well,0.0,1.375,APPLCERT,01-1127,UTAH SCHOOL AND INSTITUTIONAL TRUST LANDS ADMIN.,19971211,"Domestic,Irrigation,Other",


In [10]:
# Creating the output Dataframe for POUs.

dfPOU = pd.DataFrame(index=dfPOUin.index)

# Water Source
dfPOU["in_WaterSourceTypeCV"] = ""

# Site
dfPOU["in_Latitude"] = dfPOUin['Latitude']
dfPOU["in_Longitude"] = dfPOUin['Longitude']
dfPOU["in_PODorPOUSite"] = "POU"
dfPOU["in_SiteName"] = "Unspecified"
dfPOU["in_SiteNativeID"] = "POU" + dfPOUin['RECORD_ID'].astype(str)
dfPOU["in_SiteTypeCV"] = ""

# Allocation
dfPOU["in_AllocationFlow_CFS"] = dfPOUin['in_AllocationFlow_CFS'].astype(float)  # from POD data
dfPOU["in_AllocationVolume_AF"] = dfPOUin['in_AllocationVolume_AF'].astype(float)
dfPOU['in_AllocationLegalStatusCV'] = ""
dfPOU["in_AllocationNativeID"] = dfPOUin['WRNUM'].str.strip()
dfPOU['in_AllocationOwner'] = dfPOUin['in_AllocationOwner']  # from POD data
dfPOU['in_AllocationPriorityDate'] = dfPOUin['in_AllocationPriorityDate']  # from POD data
dfPOU["in_BeneficialUseCategory"] = dfPOUin['in_BeneficialUseCategory']  # from POD data
dfPOU["in_IrrigatedAcreage"] = dfPOUin['ACRES'].astype(float)

dfPOU = dfPOU.drop_duplicates().reset_index(drop=True)
print(len(dfPOUin))
dfPOU.head(1)

780913


,in_WaterSourceTypeCV,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_AllocationFlow_CFS,in_AllocationVolume_AF,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_BeneficialUseCategory,in_IrrigatedAcreage
0,,38.655222,-109.67395,POU,Unspecified,POU21320841,,0.0,1.375,,01-1127,UTAH SCHOOL AND INSTITUTIONAL TRUST LANDS ADMIN.,19971211,"Domestic,Irrigation,Other",0.004309


## Concatenate POD and POU Data.  Clean Data.

In [11]:
# Concatenate
frames = [dfPOD, dfPOU]
dfout = pd.concat(frames)

#Removing all NaN Values and replacing with blank
dfout = dfout.replace(np.nan, "", regex=True).reset_index()

print(len(dfout))
dfout.head(1)

674075


,index,in_WaterSourceTypeCV,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_AllocationFlow_CFS,in_AllocationVolume_AF,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_BeneficialUseCategory,in_IrrigatedAcreage
0,0,Underground,38.624341,-109.402417,POD,Non-Production Well: Test,POD795590894,Non-Production Well: Test,0.0,0.0,APPLAPP,0005007P00,CASTLE VALLEY TOWN OF,,Unspecified,


In [16]:
dfout['in_AllocationFlow_CFS'].unique()

array([0.000e+00, 1.000e-02, 5.400e-01, ..., 5.800e+01, 1.126e+00,
             nan])

In [13]:
# Clean Owner info. Remove special characters
import re

def cleanOwnerDataFunc(Val):
    Val = re.sub("[$@&.;,/\)(-]", "", Val).strip()
    return Val

dfout['in_AllocationOwner'] = dfout.apply(lambda row: cleanOwnerDataFunc(row['in_AllocationOwner']), axis=1)
dfout['in_AllocationOwner'].unique()

array(['CASTLE VALLEY TOWN OF',
       'SPANISH VALLEY WATER  SEWER IMPROVEMENT DISTRICT',
       'INTERNATIONAL URANIUM USA CORP', ..., 'BUD AND KELLI C MUNNS',
       'HORROCKS ENGINEERS', 'D ROBINSON'], dtype=object)

In [14]:
# Changing datatype of used date fields. 
dfout['in_AllocationPriorityDate'] = pd.to_datetime(dfout['in_AllocationPriorityDate'], errors = 'coerce')
dfout['in_AllocationPriorityDate'] = pd.to_datetime(dfout["in_AllocationPriorityDate"].dt.strftime('%m/%d/%Y'))

In [15]:
# Making Sure datatype of Long, Lat, Wrex, Irrigation are Float
dfout['in_AllocationFlow_CFS'] = pd.to_numeric(dfout['in_AllocationFlow_CFS'], errors='coerce')
dfout['in_AllocationVolume_AF'] = pd.to_numeric(dfout['in_AllocationVolume_AF'], errors='coerce')

In [ ]:
# Creating WaterSourceTypeCV

WaterSourceTypeCVDictionary={
"Underground" : "Groundwater",
"Abandonded Well" : "Groundwater",
"Point to Point" : "Surface Water",
"Surface" : "Surface Water",
"Return" : "Surface Water",
"Drain" : "Surface Water",
"Spring" : "Surface Water",
"Rediversion" : "Surface Water"}
def CreateWaterSourceTypeCV(val):
    if val == "" or pd.isnull(val):
        outString = "Unspecified"
    else:
        val = val.strip()
        try:
            outString = WaterSourceTypeCVDictionary[val]
        except:
            outString = "Unspecified"
    return outString

dfout['in_WaterSourceTypeCV'] = dfout.apply(lambda row: CreateWaterSourceTypeCV(row['in_WaterSourceTypeCV']), axis=1)
dfout['in_WaterSourceTypeCV'].unique()

In [ ]:
# Assign SiteTypeCV value.
# Uses the re library, but requires for loop.
# Order that the lists are inputed into dictoinary is important, want to overide generic search with a more specific search.

# Create the Lists
canalList = ["canal", "canals"]
creekList = ["creek"]
ditchList = ["ditch"]
drainList = ["drain", "drains"]
lakeList = ["lake"]
pondList = ["pond"]
reservoirList = ["reservoir"]
riverList = ["river", "fork", "surface"]
sloughList = ["slough"]
springList = ["spring", "springs", "gulch", "seep"]
tunnelList = ["tunnel", "tunnels"]
washList = ["wash"]
wellList = ["well", "wells", "well:", "draw", "hollow"]

# Making the dictionary
listDictionary = {}
listDictionary["Canal"] = canalList
listDictionary["Creek"] = creekList
listDictionary["Ditch"] = ditchList
listDictionary["Drain"] = drainList
listDictionary["Lake"] = lakeList
listDictionary["Pond"] = pondList
listDictionary["Reservoir"] = reservoirList
listDictionary["River"] = riverList
listDictionary["Slough"] = sloughList
listDictionary["Spring"] = springList
listDictionary["Tunnel"] = tunnelList
listDictionary["Wash"] = washList
listDictionary["Well"] = wellList

def CreateSiteTypeCV(val):
    if val == "" or pd.isnull(val):
        outString = "Unspecified"
    else:
        outString = "Unspecified" # Default
        
        # Cleaning text / simple search format
        val = val.replace(",", " ")
        val = val.replace(".", " ")
        val = val.replace(";", " ")
        val = val.replace("-", " ")
        val = val.replace("/", " ")
        val = val.replace("(", " ")
        val = val.replace(")", " ")
        val = val.lower().strip()
        val = " "+val+" "
        
        for x in listDictionary:
            labelString = x
            valueList = listDictionary[x]
            for words in valueList:
                if re.search(" "+words+ " ", val): outString = x
            
    return outString

dfout['in_SiteTypeCV'] = dfout.apply(lambda row: CreateSiteTypeCV( row['in_SiteTypeCV']), axis=1)
dfout['in_SiteTypeCV'].unique()

In [ ]:
# Assign LegalStatusCV value.
# Uses the re library, but requires for loop.
# Order that the lists are inputed into dictoinary is important, want to overide generic search with a more specific search.

# Create the Lists
ADECList = ["ADEC"]
ADVList = ["ADV"]
APPList = ["APP"]
CERTList = ["CERT"]
DECList = ["DEC"]
DILList = ["DIL"]
DISList = ["DIS"]
EXPList = ["EXP"]
FORFList = ["FORF"]
LAPList = ["LAP"]
NPRList = ["NPR"]
NUSEList = ["NUSE"]
PERFList = ["PERF"]
REJList = ["REJ"]
RNUMList = ["RNUM"]
STATUSList = ["STATUS"]
TEMPList = ["TEMP"]
TERMList = ["TERM"]
UGWCList = ["UGWC"]
UNAPList = ["UNAP"]
WDList = ["WD"]
WUCList = ["WUC"]


# Making the dictionary
listDictionary = {}

listDictionary["Lapsed"] = LAPList

listDictionary["Adjudication Decree"] = ADECList
listDictionary["Adverse Use Claim"] = ADVList
listDictionary["Approved"] = APPList
listDictionary["Certificated"] = CERTList
listDictionary["Decree"] = DECList
listDictionary["Diligence Claim"] = DILList
listDictionary["Disallowed"] = DISList
listDictionary["Expired"] = EXPList
listDictionary["Forfeited"] = FORFList
listDictionary["No Proof Required"] = NPRList
listDictionary["Nonuse"] = NUSEList
listDictionary["Perfected"] = PERFList
listDictionary["Rejected"] = REJList
listDictionary["Renumbered"] = RNUMList
listDictionary["Deff"] = STATUSList
listDictionary["Temp Applications"] = TEMPList
listDictionary["Terminated"] = TERMList
listDictionary["Underground Water Claim"] = UGWCList
listDictionary["Unapproved"] = UNAPList
listDictionary["Withdrawn"] = WDList
listDictionary["Water User`s Claim"] = WUCList


def CreateLegalStatus(val):
    val = str(val).strip()
    if val == "" or pd.isnull(val):
        outString = "Unspecified"
    else:
        outString = ""
        for x in listDictionary:
            valueList = listDictionary[x]
            for words in valueList:
                if words in val: outString = x
    
    if outString == "" or pd.isnull(val):
        outString = "Unspecified"

    return outString

dfout['in_AllocationLegalStatusCV'] = dfout.apply(lambda row: CreateLegalStatus( row['in_AllocationLegalStatusCV']), axis=1)
dfout['in_AllocationLegalStatusCV'].unique()

## WaDE Custom Elements (due to missing state site info)

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEUT_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = dfout['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceTypeCV'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_WaterSourceNativeID'] = dfout.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceTypeCV']), axis=1)
dfout['in_WaterSourceNativeID'].unique()

## Shapefile Data
- For attaching geometry to POU csv inputs.

In [ ]:
# PoU Shapefile Data
# Shapefile input
dfPoUshapetemp = gpd.read_file('Utah_Place_of_Use_Irrigation/Utah_Place_of_Use_Irrigation.shp')
dfPoUshapetemp.head(3)

In [ ]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['RECORD_ID'].astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

## Export Data

In [ ]:
#Removing all NaN Values and replacing with blank
dfout = dfout.replace(np.nan, "", regex=True)
print(len(dfout))
dfout.head(3)

In [ ]:
#Exporting to Finished File
dfout.to_csv('P_UtahMaster.csv', index=False)  # The output
dfPoUshape.to_csv('P_utGeometry.csv', index=False) # The output geometry.

In [ ]:
#Revewing data
import pandas as pd
dfcheck = pd.read_csv("G:/Shared drives/WaDE Data/Utah/WaterAllocation/ProcessedInputData/waterallocations.csv")
dfcheck.head(1)
dfcheck['AllocationLegalStatusCV'].unique()